In [27]:
import logging
import os
import re
from queue import Queue
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
openai_api_key = os.getenv('OPENAI_API_KEY')

# Set up logging
logging.basicConfig(level=logging.INFO)

# Define constants for evaluation criteria
MAX_EVALUATIONS = 11
EVALUATION_SCORE_THRESHOLD = 7





In [28]:
decomposition_system_template = SystemMessagePromptTemplate.from_template(
    "You are an AI that decomposes complex code generation tasks into smaller, manageable sub-tasks. Please output the decomposed plan as a detailed, markdown numbered list of steps."
)

decomposition_human_template = HumanMessagePromptTemplate.from_template(
    "Given the complex code generation task: '{task}', please decompose it into a detailed, numbered list of sub-tasks."
)



generation_system_template = SystemMessagePromptTemplate.from_template(
    "In your capacity as an AI, your task is to generate code that aligns with a given set of instructions. While developing this code, you should take into account the requirements for readability (the code should be easy to understand), efficiency (the code should be optimized for performance), and correctness (the code should accurately fulfill the intended purpose)."
)

generation_human_template = HumanMessagePromptTemplate.from_template(
    "Based on the provided instruction: {step}, your task is to generate a piece of code. The resulting code should meet the following criteria: it should be readable, allowing other developers to easily understand its logic; it should be efficient, performing the task with minimal use of resources; and it should be correct, accurately fulfilling the instruction's purpose."
)

ranking_system_template = SystemMessagePromptTemplate.from_template(
    "As an AI, your role is to evaluate and rank multiple proposed code solutions based on a set of quality metrics. The ranking should be expressed as a list of scores in a descending order, where each score is a numerical value between 0 and 10. The scores should reflect considerations such as the code's readability (how easy it is for a human to understand), correctness (whether the code accomplishes what it intends to), efficiency (how optimally the code uses resources), and overall quality. Please present the results in the format 'score : n'."
)

ranking_human_template = HumanMessagePromptTemplate.from_template(
    "Your task is to evaluate and rank the following code sequences based on their quality scores. When performing the ranking, you should consider factors such as readability (is the code easy to comprehend?), correctness (does the code do what it's supposed to do?), efficiency (how optimally does the code use resources?), and overall quality. Please evaluate each piece of code and assign it a score between 0 and 10. \n\n{generated}\n\nOnce you've assessed each code, compile the scores in a descending order (highest to lowest) in the following format: 'score : n'."
)

generation_loop_system_template = SystemMessagePromptTemplate.from_template(
    "You are an AI that develops code by taking into account not only the current instruction but also the context of previous instructions and pieces of code. The generated code should be seen as an evolution of the past codes, in direct response to the given instruction. It should be efficient, readable, and above all, correct."
)

generation_loop_human_template = HumanMessagePromptTemplate.from_template(
    """Generate code for the following instruction: {step}. 

    This task is part of a larger sequence of coding instructions, and hence, you should take into account the context of previous instructions and codes when developing your solution. 

    The relevant information from the previous stages is as follows:

   
    """
)


In [5]:
decomposition_llm = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
generation_llm = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
ranking_llm = ChatOpenAI(temperature=0.4, model_name='gpt-3.5-turbo')
search_llm = ChatOpenAI(temperature=0.2, model_name='gpt-3.5-turbo')


decomposition_chain = LLMChain(llm=decomposition_llm,prompt =ChatPromptTemplate.from_messages([decomposition_system_template, decomposition_human_template]))
generation_chain = LLMChain(llm=generation_llm, prompt=ChatPromptTemplate.from_messages([generation_system_template, generation_human_template]))
#search_chain = LLMChain(llm=search_llm, prompt=ChatPromptTemplate.from_messages([search_system_template, search_human_template]))
ranking_chain = LLMChain(llm=ranking_llm, prompt=ChatPromptTemplate.from_messages([ranking_system_template, ranking_human_template]))
generation_loop_chain = LLMChain(llm=generation_llm, prompt=ChatPromptTemplate.from_messages([generation_loop_system_template, generation_loop_human_template]))


In [6]:
ranking_chain = LLMChain(llm=ranking_llm, prompt=ChatPromptTemplate.from_messages([ranking_system_template, ranking_human_template]))

In [7]:
task = "Write a variable impedance control for force feedback using ros2, webots, webots_ros2 and ros2_control."
j = 1

In [13]:
markdown_list = decomposition_chain.run(task)

import re


# Define the regular expression pattern to match the list items
pattern = r'(\d+)\.\s+(.*)'

# Compile the regular expression pattern
regex = re.compile(pattern)

# Find all matches of the pattern in the markdown list
matches = regex.findall(markdown_list)

# Convert the matches into a dictionary
steps = {int(match[0]): match[1] for match in matches}

# Print the resulting dictionary
print(steps)



{1: '**Install necessary software packages**: Install ROS2, webots, webots_ros2, and ros2_control packages that are required for the project.', 2: '**Create a ROS2 workspace**: Create a new ROS2 workspace where all the code and packages will be stored.', 3: '**Create a ROS2 package**: Create a new ROS2 package inside the workspace, which will contain the code for the variable impedance control.', 4: '**Create a URDF model for the robot**: Create a URDF model that represents the robot that will be used in the simulation.', 5: '**Create a robot controller**: Write a robot controller that takes in a desired force and calculates the required impedance to achieve the desired force.', 6: '**Create a webots world file**: Create a webots world file that includes the robot model and a ground plane.', 7: '**Integrate ROS2 and webots**: Integrate the ROS2 and webots environments so that they can communicate with each other.', 8: '**Integrate webots and ros2_control**: Integrate the webots and ros

In [14]:
print(steps[j])

**Install necessary software packages**: Install ROS2, webots, webots_ros2, and ros2_control packages that are required for the project.


In [15]:
# Define an empty dictionary to store the generated output for each step
generated = {}

# Generate the output for step 1 four times using the LLMChain
for i in range(1, 5):
    output = generation_chain.run(steps[j])
    generated[i] = output

In [18]:
print(generated)

{1: 'To install the necessary software packages (ROS2, Webots, Webots_ROS2, and ROS2_Control) required for the project, you can follow the below steps:\n\n1. Install ROS2:\n```\nsudo apt update\nsudo apt install curl gnupg2 lsb-release\ncurl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -\nsudo sh -c \'echo "deb [arch=amd64,arm64] http://packages.ros.org/ros2/ubuntu `lsb_release -cs` main" > /etc/apt/sources.list.d/ros2-latest.list\'\nsudo apt update\nsudo apt install ros-foxy-desktop\n```\n2. Install Webots:\n```\nsudo apt update\nsudo apt install webots\n```\n3. Install Webots_ROS2:\n```\nsudo apt update\nsudo apt install ros-foxy-webots-ros2\n```\n4. Install ROS2_Control:\n```\nsudo apt update\nsudo apt install ros-foxy-ros2-control ros-foxy-ros2-controllers\n```\nNote: Make sure to source the ROS2 installation before using it. You can do this by adding the following line to your `.bashrc` file:\n```\nsource /opt/ros/foxy/setup.bash\n```\nThis 

In [17]:
# Get the highest score 
highest_score = max(ranked.keys())

# Get the code(s) for the highest score
highest_codes = ranked[highest_score]

NameError: name 'ranked' is not defined

In [29]:
# Convert dictionary to string with separator and indicators
generated_string = ""
for i, code in generated.items():
    generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

# Pass the generated code sequences to the ranking_chain
ranking = ranking_chain.run(generated_string)

In [30]:
print(generated_string)



Code 1:
To install the necessary software packages (ROS2, Webots, Webots_ROS2, and ROS2_Control) required for the project, you can follow the below steps:

1. Install ROS2:
```
sudo apt update
sudo apt install curl gnupg2 lsb-release
curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
sudo sh -c 'echo "deb [arch=amd64,arm64] http://packages.ros.org/ros2/ubuntu `lsb_release -cs` main" > /etc/apt/sources.list.d/ros2-latest.list'
sudo apt update
sudo apt install ros-foxy-desktop
```
2. Install Webots:
```
sudo apt update
sudo apt install webots
```
3. Install Webots_ROS2:
```
sudo apt update
sudo apt install ros-foxy-webots-ros2
```
4. Install ROS2_Control:
```
sudo apt update
sudo apt install ros-foxy-ros2-control ros-foxy-ros2-controllers
```
Note: Make sure to source the ROS2 installation before using it. You can do this by adding the following line to your `.bashrc` file:
```
source /opt/ros/foxy/setup.bash
```
This will make the ROS2 environm

In [31]:
# Rank the generated sequences based on their scores using the ranking_chain
#string_generated = print(generated)
ranking = ranking_chain.run(generated_string)

In [32]:
print(ranking)

After evaluating each code sequence based on the given criteria, I have assigned the following scores:

Code 1: 8
Code 2: 7
Code 3: 6
Code 4: 9

Therefore, the ranking from highest to lowest score is:

1. Code 4: 9
2. Code 1: 8
3. Code 2: 7
4. Code 3: 6

The scores were assigned based on the following factors:

- Code 1: The code is well-structured and easy to follow. The instructions are clear and concise. However, it could benefit from more detailed explanations of the installation process. The score of 8 reflects its overall quality.
- Code 2: The code is easy to read and understand, but it lacks detailed explanations of the installation process. The score of 7 reflects its overall quality.
- Code 3: The code provides clear instructions for installation, but it lacks the actual commands to install the packages. The score of 6 reflects its overall quality.
- Code 4: The code is well-structured, easy to read, and provides detailed explanations of the installation process. It is also e

In [34]:
import re

# Extract code indicators and scores from ranking string
pattern = r"Code (\d+):\s*(\d+\.?\d*)"
matches = re.findall(pattern, ranking)

# Store code indicators and scores in dictionary of lists
ranked = {}
for match in matches:
    code = int(match[0])
    score = float(match[1])
    if score not in ranked:
        ranked[score] = []
    ranked[score].append(code)

# Print the ranked dictionary
print(ranked)


{8.0: [1, 1], 7.0: [2, 2], 6.0: [3, 3], 9.0: [4, 4]}


In [35]:
# Get the highest score 
highest_score = max(ranked.keys())

# Get the code(s) for the highest score
highest_codes = ranked[highest_score]

In [36]:
prev_code = ""
prev_instruction = ""
actual_instruction = ""

In [37]:
import re
j = 1
# Extract code indicators and scores from ranking string
pattern = r"Code (\d+):\s*(\d+\.?\d*)"
matches = re.findall(pattern, ranking)

# Store code indicators and scores in dictionary of lists
ranked = {}
for match in matches:
    code = int(match[0])
    score = float(match[1])
    if score not in ranked:
        ranked[score] = []
    ranked[score].append(code)

# Get the highest score 
highest_score = max(ranked.keys())

# Get the code(s) for the highest score
highest_score = ranked[highest_score]

# Select just the first code 
highest_score = highest_score[0]

highest_code_str = generated[highest_score]



# Print the highest code 
print(f"The highest-scoring code is:\n\n {highest_code_str}")

prev_code = prev_code + highest_code_str
k = j
j = j + 1
prev_instruction = steps[k] + prev_instruction
actual_instruction = steps[j]



# highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"
# generated_new = generation_loop_chain.run(highest_code_str_memory)
# # Define an empty dictionary to store the generated output for each step
# generated_new = {}

# # Generate the output for step 1 four times using the LLMChain
# for i in range(1, 5):
#     output = generation_loop_chain.run(highest_code_str_memory)
#     generated_new[i] = output

# # Convert dictionary to string with separator and indicators
# generated_string = ""
# for i, code in generated_new.items():
#     generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

# # Pass the generated code sequences to the ranking_chain
# ranking = ranking_chain.run(generated_string)

# # Store code indicators and scores in dictionary of lists
# ranked = {}
# for match in matches:
#     code = int(match[0])
#     score = float(match[1])
#     if score not in ranked:
#         ranked[score] = []
#     ranked[score].append(code)

# # Get the highest score 
# highest_score = max(ranked.keys())

# # Get the code(s) for the highest score
# highest_score = ranked[highest_score]

# # Select just the first code 
# highest_score = highest_score[0]

# highest_code_str = generated_new[highest_score]



# prev_code = highest_code_str + prev_code
# k = j
# j = j + 1
# prev_instruction = steps[k] + prev_instruction
# actual_instruction = steps[j]

# highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"



while j <= 8:
    highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"
    generated_new = generation_loop_chain.run(highest_code_str_memory)
    # Define an empty dictionary to store the generated output for each step
    generated_new = {}

    # Generate the output for step 1 four times using the LLMChain
    for i in range(1, 5):
        output = generation_loop_chain.run(highest_code_str_memory)
        generated_new[i] = output

    # Convert dictionary to string with separator and indicators
    generated_string = ""
    for i, code in generated_new.items():
        generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

    # Pass the generated code sequences to the ranking_chain
    ranking = ranking_chain.run(generated_string)

    # Store code indicators and scores in dictionary of lists
    ranked = {}
    for match in matches:
        code = int(match[0])
        score = float(match[1])
        if score not in ranked:
            ranked[score] = []
        ranked[score].append(code)

    # Get the highest score 
    highest_score = max(ranked.keys())

    # Get the code(s) for the highest score
    highest_score = ranked[highest_score]

    # Select just the first code 
    highest_score = highest_score[0]

    highest_code_str = generated_new[highest_score]



    prev_code = prev_code + highest_code_str
    k = j
    j = j + 1
    prev_instruction = steps[k] + prev_instruction
    actual_instruction = steps[j]

    highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"
    print("##########################################################")
    print(highest_code_str_memory)

The highest-scoring code is:

 Since the installation process may differ depending on the operating system, it is necessary to determine the specific operating system being used before writing the installation code. 

Here is an example of installation code for Ubuntu 20.04:

```bash
# Install ROS2
sudo apt update
sudo apt install curl gnupg2 lsb-release
curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
sudo sh -c 'echo "deb [arch=$(dpkg --print-architecture)] http://packages.ros.org/ros2/ubuntu $(lsb_release -cs) main" > /etc/apt/sources.list.d/ros2-latest.list'
sudo apt update
sudo apt install ros-foxy-desktop

# Install Webots
sudo snap install webots

# Install webots_ros2
sudo apt install ros-foxy-webots-ros

# Install ros2_control
sudo apt install ros-foxy-ros2-control ros-foxy-ros2-controllers
```

This code installs ROS2, Webots, webots_ros2, and ros2_control packages in a readable and efficient manner. It also accurately fulfills the i

KeyboardInterrupt: 

In [118]:
import re
j = 1
# Extract code indicators and scores from ranking string
pattern = r"Code (\d+):\s*(\d+\.?\d*)"
matches = re.findall(pattern, ranking)

# Store code indicators and scores in dictionary of lists
ranked = {}
for match in matches:
    code = int(match[0])
    score = float(match[1])
    if score not in ranked:
        ranked[score] = []
    ranked[score].append(code)

# Get the highest score 
highest_score = max(ranked.keys())

# Get the code(s) for the highest score
highest_score = ranked[highest_score]

# Select just the first code 
highest_score = highest_score[0]

highest_code_str = generated[highest_score]



# Print the highest code 
print(f"The highest-scoring code is:\n\n {highest_code_str}")

prev_code = prev_code + highest_code_str
k = j
j = j + 1
prev_instruction = steps[k] + prev_instruction
actual_instruction = steps[j]



# highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"
# generated_new = generation_loop_chain.run(highest_code_str_memory)
# # Define an empty dictionary to store the generated output for each step
# generated_new = {}

# # Generate the output for step 1 four times using the LLMChain
# for i in range(1, 5):
#     output = generation_loop_chain.run(highest_code_str_memory)
#     generated_new[i] = output

# # Convert dictionary to string with separator and indicators
# generated_string = ""
# for i, code in generated_new.items():
#     generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

# # Pass the generated code sequences to the ranking_chain
# ranking = ranking_chain.run(generated_string)

# # Store code indicators and scores in dictionary of lists
# ranked = {}
# for match in matches:
#     code = int(match[0])
#     score = float(match[1])
#     if score not in ranked:
#         ranked[score] = []
#     ranked[score].append(code)

# # Get the highest score 
# highest_score = max(ranked.keys())

# # Get the code(s) for the highest score
# highest_score = ranked[highest_score]

# # Select just the first code 
# highest_score = highest_score[0]

# highest_code_str = generated_new[highest_score]



# prev_code = highest_code_str + prev_code
# k = j
# j = j + 1
# prev_instruction = steps[k] + prev_instruction
# actual_instruction = steps[j]

# highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"



while j <= 5:
    highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"
    generated_new = generation_loop_chain.run(highest_code_str_memory)
    # Define an empty dictionary to store the generated output for each step
    generated_new = {}

    # Generate the output for step 1 four times using the LLMChain
    for i in range(1, 5):
        output = generation_loop_chain.run(highest_code_str_memory)
        generated_new[i] = output

    # Convert dictionary to string with separator and indicators
    generated_string = ""
    for i, code in generated_new.items():
        generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

    # Pass the generated code sequences to the ranking_chain
    ranking = ranking_chain.run(generated_string)

    # Store code indicators and scores in dictionary of lists
    ranked = {}
    for match in matches:
        code = int(match[0])
        score = float(match[1])
        if score not in ranked:
            ranked[score] = []
        ranked[score].append(code)

    # Get the highest score 
    highest_score = max(ranked.keys())

    # Get the code(s) for the highest score
    highest_score = ranked[highest_score]

    # Select just the first code 
    highest_score = highest_score[0]

    highest_code_str = generated_new[highest_score]



    prev_code = prev_code + highest_code_str
    k = j
    j = j + 1
    prev_instruction = steps[k] + prev_instruction
    actual_instruction = steps[j]

    highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"
    print("##########################################################")
    print(highest_code_str_memory)

The highest-scoring code is:

 Here's an example code snippet that defines the impedance parameters for a control system in Python:

```
# Define impedance parameters
stiffness = 1000.0  # N/m
damping = 50.0  # N/(m/s)
inertia = 10.0  # kg

# Use the impedance parameters in your control system
def control_system(x, x_desired, x_dot_desired):
    x_dot = get_current_velocity(x)
    force = stiffness * (x_desired - x) + damping * (x_dot_desired - x_dot) + inertia * (x_dot_desired - x_dot)
    apply_force_to_system(force)
```

In this example, the impedance parameters are defined as `stiffness`, `damping`, and `inertia`. These parameters are then used in the `control_system` function to calculate the force to apply to the system based on the desired position and velocity.
##########################################################


Highest Code:
Based on the previous instruction and code, here's an implementation of the impedance control algorithm using the ROS2 API in Python:

```
import

In [119]:
print(prev_code)

To implement the functionality to apply forces and get feedback data from the Webots simulation environment, you can use the `Robot` class provided by the `controller` module in the Webots API. Here's an example code snippet in Python:

```
from controller import Robot

# Create a Webots robot instance
robot = Robot()

# Get the time step of the current world
timestep = int(robot.getBasicTimeStep())

# Get the wheels and distance sensors
left_wheel = robot.getDevice('left wheel')
right_wheel = robot.getDevice('right wheel')
ds_left = robot.getDevice('ds left')
ds_right = robot.getDevice('ds right')

# Enable sensors
ds_left.enable(timestep)
ds_right.enable(timestep)

# Set motor velocities
left_wheel.setVelocity(1.0)
right_wheel.setVelocity(1.0)

while robot.step(timestep) != -1:
    # Read sensor values
    ds_left_value = ds_left.getValue()
    ds_right_value = ds_right.getValue()

    # Apply force based on sensor values
    if ds_left_value > 100 and ds_right_value > 100:
        l

In [85]:
# Convert dictionary to string with separator and indicators
generated_string = ""
for i, code in generated_new.items():
    generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

# Pass the generated code sequences to the ranking_chain
ranking = ranking_chain.run(generated_string)

In [86]:
print(ranking)

Based on the given factors, the ranking of the code sequences from best to worst is:

1. Code 2: 9
2. Code 1: 8
3. Code 3: 6
4. Code 4: 4

Code 2 is the best because it is well-organized, easy to follow, and provides clear instructions for creating a new ROS2 package for the variable impedance control code. It also includes steps for testing the code.

Code 1 is also good, but it is more focused on the technical details of creating a new ROS2 package and does not provide as much guidance on implementing the variable impedance control code.

Code 3 is less clear and requires the user to edit the CMakeLists.txt file manually, which could lead to errors. It also does not provide instructions for testing the code.

Code 4 is the least clear and does not provide any guidance on implementing the variable impedance control code. It only adds dependencies to the package.xml file.


In [87]:
# Store code indicators and scores in dictionary of lists
ranked = {}
for match in matches:
    code = int(match[0])
    score = float(match[1])
    if score not in ranked:
        ranked[score] = []
    ranked[score].append(code)

# Get the highest score 
highest_score = max(ranked.keys())

# Get the code(s) for the highest score
highest_score = ranked[highest_score]

# Select just the first code 
highest_score = highest_score[0]

highest_code_str = generated_new[highest_score]

In [89]:
print(f"The highest-scoring code is:\n\n {highest_code_str}")

prev_code = highest_code_str + prev_code
k = j
j = j + 1
prev_instruction = steps[k] + prev_instruction
actual_instruction = steps[j]

highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"

The highest-scoring code is:

 To create a new ROS2 package for the variable impedance control code, follow these steps:

1. Open a terminal and navigate to your ROS2 workspace directory.
2. Run the following command to create a new package named "variable_impedance_control":

   ```
   ros2 pkg create --build-type ament_cmake variable_impedance_control
   ```

3. This will create the necessary directory structure and files for the package. Navigate to the new package directory by running:

   ```
   cd variable_impedance_control
   ```

4. Inside the package directory, create a new directory called "src" to store your source code:

   ```
   mkdir src
   ```

5. Create a new source code file for your variable impedance control code inside the "src" directory. You can use any programming language supported by ROS2, such as C++ or Python.

6. Once you have written your code, you will need to modify the CMakeLists.txt file in the package directory to build your code. Add the following li

In [90]:
generated_new = generation_loop_chain.run(highest_code_str_memory)
# Define an empty dictionary to store the generated output for each step
generated_new = {}

# Generate the output for step 1 four times using the LLMChain
for i in range(1, 5):
    output = generation_loop_chain.run(highest_code_str_memory)
    generated_new[i] = output

In [92]:
# Rank the generated sequences based on their scores using the ranking_chain
#string_generated = print(generated)
# Convert dictionary to string with separator and indicators
generated_string = ""
for i, code in generated_new.items():
    generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

# Pass the generated code sequences to the ranking_chain
ranking = ranking_chain.run(generated_string)


In [98]:
print(f"The highest-scoring code is:\n\n {highest_code_str}")

prev_code =   prev_code + highest_code_str_memory
k = j
j = j + 1
prev_instruction = steps[k] + prev_instruction
actual_instruction = steps[j]

highest_code_str_memory = f"\n\n{'='*50}\nHighest Code:\n{highest_code_str}\n\n{'='*50}\nPrevious Code:\n{prev_code}\n\n{'='*50}\nPrevious Instruction:\n{prev_instruction}n\n{'='*50}\nActual Instruction:\n{actual_instruction}"


generated_new = generation_loop_chain.run(highest_code_str_memory)
# Define an empty dictionary to store the generated output for each step
generated_new = {}

# Generate the output for step 1 four times using the LLMChain
for i in range(1, 5):
    output = generation_loop_chain.run(highest_code_str_memory)
    generated_new[i] = output


# Rank the generated sequences based on their scores using the ranking_chain
#string_generated = print(generated)
# Convert dictionary to string with separator and indicators
generated_string = ""
for i, code in generated_new.items():
    generated_string += f"\n\n{'='*50}\nCode {i}:\n{code}"

# Pass the generated code sequences to the ranking_chain
ranking = ranking_chain.run(generated_string)


The highest-scoring code is:

 To create a new ROS2 package for the variable impedance control code, follow these steps:

1. Open a terminal and navigate to your ROS2 workspace directory.
2. Run the following command to create a new package named "variable_impedance_control":

   ```
   ros2 pkg create --build-type ament_cmake variable_impedance_control
   ```

3. This will create the necessary directory structure and files for the package. Navigate to the new package directory by running:

   ```
   cd variable_impedance_control
   ```

4. Inside the package directory, create a new directory called "src" to store your source code:

   ```
   mkdir src
   ```

5. Create a new source code file for your variable impedance control code inside the "src" directory. You can use any programming language supported by ROS2, such as C++ or Python.

6. Once you have written your code, you will need to modify the CMakeLists.txt file in the package directory to build your code. Add the following li

In [104]:
#print(f"The highest-scoring code is:\n\n {ranking}")
print(highest_code_str_memory)



Highest Code:
To create a new ROS2 package for the variable impedance control code, follow these steps:

1. Open a terminal and navigate to your ROS2 workspace directory.
2. Run the following command to create a new package named "variable_impedance_control":

   ```
   ros2 pkg create --build-type ament_cmake variable_impedance_control
   ```

3. This will create the necessary directory structure and files for the package. Navigate to the new package directory by running:

   ```
   cd variable_impedance_control
   ```

4. Inside the package directory, create a new directory called "src" to store your source code:

   ```
   mkdir src
   ```

5. Create a new source code file for your variable impedance control code inside the "src" directory. You can use any programming language supported by ROS2, such as C++ or Python.

6. Once you have written your code, you will need to modify the CMakeLists.txt file in the package directory to build your code. Add the following lines to the end 

In [ ]:

decomposition_llm = ChatOpenAI(temperature=1, model_name='gpt-3.5-turbo')
generation_llm = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
ranking_llm = ChatOpenAI(temperature=0.5, model_name='gpt-3.5-turbo')
search_llm = ChatOpenAI(temperature=0.2, model_name='gpt-3.5-turbo')

decomposition_chain = LLMChain(llm=decomposition_llm,prompt =ChatPromptTemplate.from_messages([decomposition_system_template, decomposition_human_template]))
generation_chain = LLMChain(llm=generation_llm, prompt=ChatPromptTemplate.from_messages([generation_system_template, generation_human_template]))
#search_chain = LLMChain(llm=search_llm, prompt=ChatPromptTemplate.from_messages([search_system_template, search_human_template]))
ranking_chain = LLMChain(llm=ranking_llm, prompt=ChatPromptTemplate.from_messages([ranking_system_template, ranking_human_template]))
generation_loop_chain = LLMChain(llm=generation_llm, prompt=ChatPromptTemplate.from_messages([generation_loop_system_template, generation_loop_human_template]))


In [38]:
# # Define an empty dictionary to store the generated output for each step
# generated = {}

# # Loop through all the steps and generate the output for each one
# for step in steps:
#     # Generate the output for the current step using the LLMChain
#     output = generation_chain.run(steps[step])
    
#     # Add the output to the generated dictionary with the step number as the key
#     generated[step] = output

In [ ]:
# Define a regular expression pattern for evaluation score
evaluation_pattern = r"score\s*:\s*(\d+)"

# Compile the regular expression pattern
regex = re.compile(evaluation_pattern)

# Find all matches of the pattern in the evaluated string values
matches = [regex.findall(value) for value in evaluated.values()]

# Flatten the list of matches
matches = [match for sublist in matches for match in sublist]

# Convert the matches into a dictionary
grades = {i+1: int(match) for i, match in enumerate(matches)}

# Print the resulting dictionary
print(grades)

In [49]:
# Define a regular expression pattern for evaluation score
evaluation_pattern = r"score\s*:\s*(\d+)"

# Compile the regular expression pattern
regex = re.compile(evaluation_pattern)

# Find all matches of the pattern in the evaluated string
matches = regex.findall(evaluated)

# Convert the matches into a dictionary
grades = {i+1: int(match) for i, match in enumerate(matches)}

# Print the resulting dictionary
print(grades)

TypeError: expected string or bytes-like object

In [ ]:
def validate_task(task):
    if not task or not isinstance(task, str) or len(task.strip()) == 0:
        logging.error("Invalid task. Please provide a non-empty string.")
        return False
    return True

def process_task(task, queue, stack, code, evaluation_counter):
    try:
        print(f"Decomposing task: {task}")
        decomposition = decomposition_chain(task)
        print(decomposition)

        # split the decomposition text by newline characters
        steps = decomposition["text"].split("\n")

        for step in steps:
            print(f"Generating next step for part: {step}")
            generation = generation_chain(step)

            if not generation["text"].strip():  # If no code is generated, raise ValueError
                raise ValueError("No code generated!")
            
            print(f"Evaluating generated code: {generation}")
            evaluation_score = evaluation_chain({"code": generation["text"]})  # Pass the generated code as an argument

            evaluation_counter += 1
            
            if evaluation_score >= EVALUATION_SCORE_THRESHOLD:
                print(f"Generated code meets the threshold. Added to queue and stack.")
                queue.put(generation)
                stack.append(generation)
            elif stack:
                print(f"Generated code doesn't meet the threshold. Reverting to last state in stack.")
                last_state = stack.pop()
                queue.put(last_state)
        
        if not queue.empty():
            print(f"Choosing next step from the proposed steps in the queue.")
            search = search_chain(queue.queue)
            code += search
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")

    queue.queue.clear()
    stack.clear()

    return code, evaluation_counter

def main():
    task = 'Write a variable impedance control code for force feedback using ros2, webots and webots_ros2'
    
    if not validate_task(task):
        return

    queue = Queue()
    stack = []
    
    queue.put(task)
    stack.append(task)

    code = """"""
    
    evaluation_counter = 0

    while not queue.empty():
        task = queue.get()
        
        code, evaluation_counter = process_task(task, queue, stack, code, evaluation_counter)

        if  evaluation_counter >= MAX_EVALUATIONS:
            break
    
    logging.info(f"Final code: {code}")

if __name__ == "__main__":
    main()
